In [72]:
import scipy.io
import pandas as pd
import networkx as nx
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
name = 'mutag'
nodel = 1
l = 3
num_eig = 3

In [3]:
dataset = scipy.io.loadmat('datasets/'+name+'.mat')
names = list(dataset.keys())[3:]
names


['lmutag', 'mutag']

In [90]:
Graphs = dataset[names[1]][0]
labels = Graphs[0][1][0][0][0].flatten()
total = len(Graphs[0])
labels

array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 6., 7., 7.])

In [97]:
def process_labels(lab):
    unique = np.unique(lab.flatten())
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(unique.reshape(-1, 1))
    oneHot = enc.transform(lab.reshape(-1, 1)).toarray()
    return unique, oneHot
process_labels(labels)

(array([3., 6., 7.]),
 array([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.]]))

In [5]:
G = nx.from_numpy_matrix(Graphs[0][0])
#d es el arreglo de grados del grafo
d = []
for i in range(len(G)):
    d.append(G.degree[i])
d = np.array(d)
#D es la matriz de grados diagonales
D = np.diag(d)
#A es la matriz de adjacencia del grafo
A = nx.adjacency_matrix(G).todense()

/tmp/ipykernel_8218/3421487673.py:10: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G).todense()


In [6]:
#M es la matriz de markov del grafo
M = np.linalg.inv(D) * A

In [35]:
#np.transpose(d)
# m es la cantidad de arcos del grafo
m = len(G.edges)
#pi es el vector de distribución estacionaria
pi = np.asmatrix(d).transpose() / (2 * m)
#PI es la matriz diagonal de distribución estacionaria
PI = np.diag(pi.transpose().tolist()[0])

In [8]:
def ro(t, P, p, Me):
    return P * np.linalg.matrix_power(Me, t) - pi.transpose() * pi
print(PI.shape, pi.shape, M.shape)
ro(1, PI, pi, M)

(23, 23) (23, 1) (23, 23)


matrix([[-0.04663923, -0.02812071, -0.04663923, -0.04663923, -0.04663923,
         -0.04663923, -0.04663923, -0.04663923, -0.04663923, -0.04663923,
         -0.04663923, -0.04663923, -0.04663923, -0.02812071, -0.04663923,
         -0.04663923, -0.04663923, -0.04663923, -0.04663923, -0.04663923,
         -0.04663923, -0.04663923, -0.04663923],
        [-0.02812071, -0.04663923, -0.02812071, -0.04663923, -0.04663923,
         -0.04663923, -0.04663923, -0.04663923, -0.04663923, -0.04663923,
         -0.04663923, -0.04663923, -0.04663923, -0.04663923, -0.04663923,
         -0.04663923, -0.04663923, -0.04663923, -0.04663923, -0.04663923,
         -0.04663923, -0.04663923, -0.04663923],
        [-0.04663923, -0.02812071, -0.04663923, -0.02812071, -0.04663923,
         -0.04663923, -0.04663923, -0.04663923, -0.04663923, -0.04663923,
         -0.04663923, -0.02812071, -0.04663923, -0.04663923, -0.04663923,
         -0.04663923, -0.04663923, -0.04663923, -0.04663923, -0.04663923,
         -0.04

In [43]:
#covarianza para v 3.3
def attr_cov(v, cova, lag):
    cov = []
    for t in range(lag):
        cov.append((v.transpose() * cova[t] * v).tolist()[0][0])
    return cov

In [26]:
#covarianzar de variable categorica 3.6
def covarianza(Me, p, P, Ha, lag):
    # cov son las matrices de covarianza
    # aut son las matrices de autocovarianza
    # tr son las trazas de aut
    cov = []
    aut = []
    tr = []
    for t in range(lag):
        cov.append(ro(t, P, p, Me))
        aut.append(Ha.transpose() * ros[-1] * Ha)
        tr.append(np.trace(aut[-1]))
    return cov, aut, tr

In [25]:
evalues, evectors = np.linalg.eigh(M.transpose(), 'L')

In [30]:
#feature1: multi-hop assortativities of node IDs, setting H = I in Eq. (3.6) for 0, 1, ... , t hops
#H es la matriz identidad del tamaño del grafo
H = np.identity(len(d))
cov, aut, res1 = covarianza(M, pi, PI, H, l)
res1

[-0.6467764060356653, -0.6467764060356653, -0.6467764060356653]

In [45]:
#feature2: average of first p dominant left eigenvector of M, that is {πv : v is a dominant eigenvector of M}
res2 = attr_cov(pi, cov, l)

[-0.044352994970278906, -0.04445460549712949, -0.04442496909346475]

In [48]:
#feature3: multi-hop assortativities of the first p dominant left eigenvectors of M (Eq. 3.3) for 0, 1, ... , t hops
h2 = evectors[:,-2]
res3 = attr_cov(h2, cov, l)

[0.006902972717857179, 0.0025986355520744246, -0.0008785154335105285]

In [50]:
#feature3: multi-hop assortativities of the first p dominant left eigenvectors of M (Eq. 3.3) for 0, 1, ... , t hops
h2 = evectors[:,-2]
res3 = attr_cov(h2, cov, l)
h3 = evectors[:,-3]
res4 = attr_cov(h3, cov, l)

[0.04324539739958903, 0.03662871219442858, 0.03192505368451449]

In [119]:
def avg_feats(Ha, p):
    print(p)
    avg = []
    for i in range(len(Ha[0])):
        avg.append(p * np.asmatrix(Ha[:,i]).transpose())
    return avg

In [120]:

if nodel == 1:
    #print(labels)
    unique, categorias = process_labels(labels)
    #print(categorias)
    cov, aut, tr = covarianza(M, pi, PI, categorias, l)
    #feature4: (If available) average of categorical metadata node attributes: {πhi : H = [h1, h2, ..hk ], 1 ≤ i ≤ k}
    res5 = np.array([np.diag(i) for i in aut]).flatten()
    #feature5: (If available) multi-hop assortativities of categorical metadata node attributes (Eq. 3.4) for 0, 1, ... , t hops
    res6 = avg_feats(categorias, pi.transpose())
    

[[0.03703704 0.03703704 0.05555556 0.03703704 0.03703704 0.05555556
  0.05555556 0.03703704 0.03703704 0.05555556 0.05555556 0.05555556
  0.05555556 0.03703704 0.05555556 0.05555556 0.03703704 0.03703704
  0.03703704 0.03703704 0.05555556 0.01851852 0.01851852]]
[matrix([[0.90740741]]), matrix([[0.05555556]]), matrix([[0.03703704]])]
